# Update distortion CDP version 8B.05.00 to version 8B.05.01

In [1]:
import datetime
import numpy as np
from astropy.io import fits
import numpy.lib.recfunctions as rec

import warnings
warnings.simplefilter('ignore')

In [2]:
# Define paths to data
workDir   = '/Users/ioannisa/Desktop/python/miri_devel/'
cdpDir    = workDir+'cdp_data/'
outDir    = cdpDir+"CDP8/"

In [4]:
# create all new distortion cdps
for miri_setup in ['IFUSHORT_12SHORT','IFUSHORT_12MEDIUM','IFUSHORT_12LONG','IFULONG_34SHORT','IFULONG_34MEDIUM','IFULONG_34LONG']:
    # open slice map pseudo-cdp
    hdulist_slicemap = fits.open(cdpDir+'CDP8/MIRI_FM_MIR{}_SLICEMAP_8B.00.00.fits'.format(miri_setup))
    
    # initialize cube placeholder
    new_sliceMaps = np.zeros((9,1024,1032)) # Make a 1032x1024 array with a total of nine layers (different transmissions)
    
    # create slice map cube
    layer = -1
    for transmission in ['10','20','30','40','50','60','70','80','90']:
        layer +=1
        new_sliceMaps[layer,:,:] = hdulist_slicemap[layer+1].data
        
    # load old distortion cdp fits file
    old_distortion_cdp = fits.open(cdpDir+'CDP8/MIRI_FM_MIR{}_DISTORTION_8B.05.00.fits'.format(miri_setup))
    
    # change headers
    old_distortion_cdp[0].header['DATE']     = datetime.datetime.utcnow().isoformat()
    old_distortion_cdp[0].header['FILENAME'] = 'MIRI_FM_MIR{}_DISTORTION_8B.05.01.fits'.format(miri_setup)
    old_distortion_cdp[0].header['VERSION']  = '8B.05.01'
    old_distortion_cdp[0].header['ORIGFILE'] = 'MIRI_FM_MIR{}_DISTORTION_8B.05.00.fits'.format(miri_setup)
    old_distortion_cdp[0].header['USEAFTER'] = "2000-01-01T00:00:00"
    old_distortion_cdp[0].header.add_history("Corrected edge slice boundaries")
    
    old_distortion_cdp['Slice_number'].data   = new_sliceMaps.copy()
    
    for level in range(1,10):
        old_distortion_cdp['Slice_number'].header['LEVEL_{}'.format(str(level))] = '{}% transmission slice map'.format(str(int(level*10.)))

    # save output
    old_distortion_cdp.writeto(outDir + "MIRI_FM_MIR{}_DISTORTION_8B.05.01.fits".format(miri_setup),overwrite=True)